In [2]:
import pandas as pd
import numpy as np


ARTICULATION:

statistical analysis on features:
df=pd.read_csv('/home/vboxuser/VoiceFiles/Voice4PD-MSA-Tool-global_V2/ArticulationData/stat_ar/Articulationfeaturesst_train_stat.csv')
#train and test data:
df_train=pd.read_csv('/home/vboxuser/VoiceFiles/Voice4PD-MSA-Tool-global_V2/ArticulationData/articulationfeaturesst_seg_dev.csv')
df_test=pd.read_csv('/home/vboxuser/VoiceFiles/Voice4PD-MSA-Tool-global_V2/ArticulationData/articulationfeaturesst_seg_test.csv')

In [25]:
#PHONATION:

#statistical analysis on features:
df=pd.read_csv('/home/vboxuser/VoiceFiles/Voice4PD-MSA-Tool-global_V2/PhonationData/stat_ph/phonationfeaturesst_seg_train_stat.csv')
#train and test data:
df_train=pd.read_csv('/home/vboxuser/VoiceFiles/Voice4PD-MSA-Tool-global_V2/PhonationData/phonationfeaturesst_seg_dev.csv')
df_test=pd.read_csv('/home/vboxuser/VoiceFiles/Voice4PD-MSA-Tool-global_V2/PhonationData/phonationfeaturesst_seg_test.csv')

PROSODY:

#statistical analysis on features:
df=pd.read_csv('/home/vboxuser/VoiceFiles/Voice4PD-MSA-Tool-global_V2/ProsodyData/stat_pr/prosodyfeaturesst_seg_train_stat.csv')
#train and test data:
df_train=pd.read_csv('/home/vboxuser/VoiceFiles/Voice4PD-MSA-Tool-global_V2/ProsodyData/prosodyfeaturesst_seg_dev.csv')
df_test=pd.read_csv('/home/vboxuser/VoiceFiles/Voice4PD-MSA-Tool-global_V2/ProsodyData/prosodyfeaturesst_seg_test.csv')

In [26]:
#from the {ImageFolder..} file obtained earlier from feature_analysis we take those values that have p_val<0.05 and effect_size>|0.4|
req_features=df[(df['p_HC_DP']<0.05) | (abs(df['p_HC_DP.1'])>0.4)]

#### 2. Obtain the selected good features as a list:

feature_list=req_features['Unnamed: 0'].tolist()
#feature_list=req_features['feature'].tolist()

#### 3. Obtaining X_train & Y_train:

df_train=df_train.fillna(0)

#replace string values to 0s & 1s in target column
df_train.replace({'Group':{"HC":0,"DP":1}},inplace=True)

X_train=df_train[df_train.columns.intersection(feature_list)]
Y_train=df_train['Group']

##### 4.1. Upsampling the data
! pip install imblearn --quiet

from imblearn.over_sampling import SMOTE

sm = SMOTE(sampling_strategy='minority', random_state=42)

X_train_resampled, Y_train_resampled = sm.fit_resample(X_train,Y_train)
df_train_upsampled = pd.concat([pd.DataFrame(Y_train_resampled), pd.DataFrame(X_train_resampled)], axis=1)


##### 4.2. Normalizing the data

import Normalization as n

X=n.normalize_train(X_train_resampled)

Y=Y_train_resampled

#checking if normalized data has a mean~0 & standard deviation~1:
print(np.mean(X))
print(np.std(X))




3.1949687866984904e-17
1.0


In [27]:
df_test=df_test.fillna(0)

X_test=df_test[df_test.columns.intersection(feature_list)]
df_test.replace({'Group':{"HC":0,"DP":1}},inplace=True)
X_test=n.normalize_test(X_test)
Y_test=df_test['Group']


In [28]:
from sklearn.decomposition import PCA

pca=PCA(n_components=4)

x_trainpca=pca.fit_transform(X)
x_testpca=pca.transform(X_test)

##convert principal components for each sample from numpy array to df:
pca_df=pd.DataFrame(x_trainpca,columns=["PCA1","PCA2","PCA3","PCA4"])
#pca_df['subject','class']=[df_train['subject'],Y]
#pca_df['class']=Y
##pca_df['subject']=df_train['subject']
pca_df.head()

#print("evr is {}".format(pca.explained_variance_ratio_))

pca_df.to_csv('/home/vboxuser/VoiceFiles/pho_train_pca.csv')

In [24]:
##convert principal components for each sample from numpy array to df:
pca_df=pd.DataFrame(x_testpca,columns=["PCA1","PCA2","PCA3","PCA4"])
#pca_df['subject','class']=[df_train['subject'],Y]
#pca_df['class']=Y_test
#pca_df['subject']=df_test['subject']
pca_df.head()

#print("evr is {}".format(pca.explained_variance_ratio_))

pca_df.to_csv('/home/vboxuser/VoiceFiles/pro_test_pca.csv')

In [67]:
pho=pd.read_csv('/home/vboxuser/VoiceFiles/pho_train_pca.csv')
pro=pd.read_csv('/home/vboxuser/VoiceFiles/pro_train_pca.csv')
art=pd.read_csv('/home/vboxuser/VoiceFiles/art_train_pca.csv')       

In [68]:
new={'PCA1':'pho1','PCA2':'pho2','PCA3':'pho3','PCA4':'pho4'}
pho.rename(columns=new,inplace=True)


new={'PCA1':'pro1','PCA2':'pro2','PCA3':'pro3','PCA4':'pro4','PCA5':'pro5'}
pro.rename(columns=new,inplace=True)


new={'PCA1':'art1','PCA2':'art2','PCA3':'art3','PCA4':'art4','PCA4.1':'art5','PCA4.2':'art6','PCA7':'art7'}
art.rename(columns=new,inplace=True)



df=pd.merge(pho,pro,on='Unnamed: 0')
df

cluster_df=pd.merge(df,art,on='Unnamed: 0')
new={'Unnamed: 0':'person_idx'}
cluster_df.rename(columns=new,inplace=True)

In [96]:
cluster_df

,person_idx,pho1,pho2,pho3,pho4,pro1,pro2,pro3,pro4,pro5,art1,art2,art3,art4,art5,art6,art7,class,subject
0,0,-0.914522,0.264392,1.847232,0.431064,-0.092288,-0.450454,-0.531557,-0.268361,0.342859,-0.341421,3.714981,1.610787,0.257709,-1.657040,-1.422633,-3.066713,0,300_P_0.wav
1,1,-1.522688,4.260832,-2.561794,0.265197,-6.267187,-0.376988,-1.201919,0.367356,-0.934877,-8.471369,9.837467,1.491241,1.816599,-0.013799,-1.814152,0.539966,0,300_P_1.wav
2,2,-1.285786,2.301453,-0.932657,1.352581,0.459633,-2.666980,1.689219,-0.655546,1.135437,-5.993691,9.331194,2.695152,1.221687,-2.323096,1.162139,0.345121,0,300_P_10.wav
3,3,-1.751591,1.304877,-0.149614,-0.310013,0.151112,-0.452275,-0.384751,-1.243988,2.714163,6.210829,8.599816,3.383598,-1.326979,-2.582407,-1.606297,-1.005032,0,300_P_11.wav
4,4,-0.510701,2.197447,0.556090,-0.436981,-0.477250,0.178098,0.312693,-1.446876,1.183369,4.544794,5.133942,2.050804,0.725321,-1.099820,2.305122,-5.579191,0,300_P_12.wav
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7751,7751,-1.590409,-1.179619,-0.406889,-0.892259,0.386461,-1.196920,-0.367881,-1.128252,0.532191,4.544941,2.273859,-0.087461,-1.579268,-3.969542,0.335890,1.436058,1,NaN
7752,7752,-0.682799,-1.008235,-1.376940,0.120193,1.390540,1.499100,-1.539248,0.552321,1.243438,-0.417336,-3.271412,-0.155757,-4.441592,-1.053259,0.194142,-1.752080,1,NaN
7753,7753,2.455026,3.886426,-1.776883,1.464813,-0.051423,-0.844687,-0.422246,-2.244149,1.017924,1.370368,1.131580,2.737765,-0.163759,-2.574562,-0.546238,0.426246,1,NaN
7754,7754,-1.714916,0.449155,-1.264943,-0.006664,1.909776,-0.977830,-1.166560,1.301498,0.402396,0.499728,-0.176930,1.134612,-3.262456,1.352663,-1.202173,1.599059,1,NaN


TEST DATAFRAME:

In [70]:
phot=pd.read_csv('/home/vboxuser/VoiceFiles/pho_test_pca.csv')
prot=pd.read_csv('/home/vboxuser/VoiceFiles/pro_test_pca.csv')
artt=pd.read_csv('/home/vboxuser/VoiceFiles/art_test_pca.csv')      

In [71]:
new={'PCA1':'pho1','PCA2':'pho2','PCA3':'pho3','PCA4':'pho4'}
phot.rename(columns=new,inplace=True)


new={'PCA1':'pro1','PCA2':'pro2','PCA3':'pro3','PCA4':'pro4','PCA5':'pro5'}
prot.rename(columns=new,inplace=True)


new={'PCA1':'art1','PCA2':'art2','PCA3':'art3','PCA4':'art4','PCA4.1':'art5','PCA4.2':'art6','PCA7':'art7'}
artt.rename(columns=new,inplace=True)



dft=pd.merge(phot,prot,on='Unnamed: 0')


test_df=pd.merge(dft,artt,on='Unnamed: 0')
new={'Unnamed: 0':'person_idx'}
test_df.rename(columns=new,inplace=True)

In [97]:
test_df

,person_idx,pho1,pho2,pho3,pho4,pro1,pro2,pro3,pro4,pro5,art1,art2,art3,art4,art5,art6,art7,class,subject
0,0,1.024856,0.916120,7.825055,2.136563,-0.809801,0.211184,0.275256,-1.563998,-0.693898,7.197059,0.370966,-0.248514,-1.586014,-0.811203,-7.172273,1.432288,0,600_AUDIO_0.wav
1,1,0.839322,-0.576252,0.412783,-2.752986,-0.341142,-1.030860,-0.230314,0.189093,0.477433,6.241912,-2.922456,2.248419,1.779566,0.947185,-5.729531,-1.164300,0,600_AUDIO_1.wav
2,2,0.459176,0.809562,-0.774536,-0.558275,-0.129159,0.550343,1.812482,-0.728252,0.435623,0.564643,-0.727580,-1.245022,-2.312372,0.228797,-0.516860,-0.250636,0,600_AUDIO_10.wav
3,3,-0.424547,1.553743,-0.750000,3.133483,0.984338,0.857242,-0.429438,-0.763521,-0.845193,2.063497,1.467360,-2.114958,0.276122,2.730262,-0.883443,2.194942,0,600_AUDIO_11.wav
4,4,3.777877,-0.121677,3.319333,-1.179350,0.297275,0.699730,-1.187971,-1.371023,0.928513,4.338600,3.030945,0.216606,-0.933515,0.567810,-0.393359,1.643880,0,600_AUDIO_12.wav
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4575,4575,-0.990623,-0.434759,-0.154011,-2.790075,-6.229763,0.508902,0.004540,0.272593,-1.304365,9.028329,-4.004783,4.530528,0.176904,-3.539882,1.437210,-0.612176,0,718_AUDIO_95.wav
4576,4576,-2.880529,0.461351,-1.416196,0.888022,2.267018,-2.546689,1.386833,-0.019531,-0.204380,4.978294,-0.865032,4.707120,3.864807,-0.429484,-0.994578,-0.321786,0,718_AUDIO_96.wav
4577,4577,-1.281973,0.557964,-1.553737,0.429263,-6.301972,-0.673068,-1.415972,0.919858,-0.182209,10.303454,-1.523913,2.898894,1.486413,-0.169954,-0.668764,5.389894,0,718_AUDIO_97.wav
4578,4578,-2.167807,-2.034239,0.467807,-1.257728,-0.230608,-1.736740,-0.308778,1.087766,-0.477219,2.275228,-3.454913,-0.222880,0.776668,0.581139,-0.336817,3.182001,0,718_AUDIO_98.wav


PRE PROCESSING:(train)

In [98]:
X_train=cluster_df.drop(['person_idx','art7','art6','art5','pro5','art4','pro4','class','subject'],axis=1)
Y_train=cluster_df['class']

In [99]:
cluster_df['class'].value_counts()

0    3878
1    3878
Name: class, dtype: int64

In [100]:
import Normalization as n

X=n.normalize_train(X_train)

print(np.mean(X))
print(np.std(X))

7.14573664121813e-18
1.0000000000000002


PRE PROCESSING:(test)

In [101]:
X_test=test_df.drop(['person_idx','art7','art6','art5','pro5','art4','pro4','class','subject'],axis=1)
Y_test=test_df['class']

In [102]:
X_test=n.normalize_test(X_test)

print(np.mean(X_test))
print(np.std(X_test))

0.02818439414167981
1.1206281833743446


In [103]:
from sklearn.svm import SVC

SVM_model = SVC(kernel='rbf',C=1,gamma=0.01, class_weight='balanced')
SVM_model.fit(X,Y)
Y_predict_SVM=SVM_model.predict(X_test)

In [104]:
from sklearn import metrics
acc= metrics.accuracy_score(Y_test,Y_predict_SVM)
acc

0.4720524017467249

In [105]:
from sklearn.metrics import classification_report
print(classification_report(Y_test,Y_predict_SVM))

              precision    recall  f1-score   support

           0       0.80      0.36      0.50      3311
           1       0.31      0.77      0.45      1269

    accuracy                           0.47      4580
   macro avg       0.56      0.56      0.47      4580
weighted avg       0.67      0.47      0.48      4580



In [79]:
features=X_train.columns

import itertools
flist=[]
for l in range(len(features)):
    for subset in itertools.combinations(features, l):
        flist.append(subset)
        #len(feature_list)

In [80]:
len(flist)

4095

In [84]:
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import confusion_matrix

class classification_models:
    def SVM_model(self,X,Y,X1,Y1):
        self.X=X
        self.Y=Y
        self.X1=X1
        self.Y1=Y1
        SVM_model = SVC(kernel='rbf',C=1, gamma=0.1, class_weight='balanced')
        SVM_model.fit(X,Y)
        Y_predict_SVM=SVM_model.predict(X1)
        acc= metrics.accuracy_score(Y1,Y_predict_SVM)
        
        tn,fp,fn,tp=confusion_matrix(Y1,Y_predict_SVM).ravel()
        P=tp/(tp+fp)
        R=tp/(tp+fn)
        #if acc>0.5:
        return [acc,P,R]
        

In [89]:
from tqdm import tqdm
i=0
result=[]
#for j in tqdm(range(1,len(flist))):
    #pass
for i in range(1,len(flist)):
    Xf_train=cluster_df[cluster_df.columns.intersection(flist[i])]
    Xf=n.normalize_train(Xf_train)
    Xf_test=test_df[test_df.columns.intersection(flist[i])]
    Xf_test=n.normalize_test(Xf_test)        

    obj=classification_models()
    a,p,r=obj.SVM_model(Xf,Y,Xf_test,Y_test)
    if a>0.50:
        res=[a,p,r]
        score=[str((flist[i])),a,p,r]
    #if res is not None:
        print(i,flist[i])
        print(res)
        
        result.append(score)
        
        
#res_df.to_csv("feature scores.csv",index=False)


2 ('pho2',)
[0.6331877729257642, 0.3214596003475239, 0.2915681639085894]
5 ('pro1',)
[0.6705240174672489, 0.14497041420118342, 0.038613081166272656]
15 ('pho1', 'pho4')
[0.508296943231441, 0.2939203354297694, 0.5524034672970843]
20 ('pho1', 'art1')
[0.506113537117904, 0.2945800579230451, 0.5610717100078803]
24 ('pho2', 'pho3')
[0.6574235807860263, 0.3568702290076336, 0.29472025216706066]
26 ('pho2', 'pro1')
[0.6100436681222707, 0.2936951316839585, 0.2899921197793538]
28 ('pho2', 'pro3')
[0.6375545851528385, 0.3301476976542137, 0.2994483845547675]
30 ('pho2', 'art1')
[0.604585152838428, 0.3398345153664303, 0.45311268715524033]
32 ('pho2', 'art3')
[0.6397379912663755, 0.34868943606036534, 0.3459416863672183]
47 ('pho4', 'art1')
[0.5080786026200873, 0.3123569794050343, 0.6453900709219859]
54 ('pro1', 'art1')
[0.5165938864628821, 0.28610230873698506, 0.4980299448384555]
85 ('pho1', 'pho2', 'art1')
[0.524235807860262, 0.3042168674698795, 0.5571315996847912]
98 ('pho1', 'pho4', 'pro1')
[0.50

503 ('pho2', 'pho4', 'pro1', 'art1')
[0.5277292576419214, 0.30682800345721695, 0.5594956658786446]
504 ('pho2', 'pho4', 'pro1', 'art2')
[0.511353711790393, 0.32336857455340867, 0.6989755713159969]
514 ('pho2', 'pho4', 'pro3', 'art1')
[0.5039301310043668, 0.3095328522597797, 0.6422379826635146]
523 ('pho2', 'pho4', 'art1', 'art3')
[0.511353711790393, 0.3127174333204484, 0.6375098502758078]
526 ('pho2', 'pho4', 'art2', 'art4')
[0.5093886462882096, 0.308984375, 0.6233254531126872]
535 ('pho2', 'pro1', 'pro3', 'art1')
[0.5558951965065502, 0.29686670207116306, 0.4405043341213554]
537 ('pho2', 'pro1', 'pro3', 'art3')
[0.5731441048034934, 0.2858926342072409, 0.3609141055949567]
544 ('pho2', 'pro1', 'art1', 'art3')
[0.5471615720524018, 0.3022113022113022, 0.4846335697399527]
569 ('pho2', 'pro3', 'art1', 'art3')
[0.5200873362445415, 0.3060542797494781, 0.5776201733648542]
587 ('pho3', 'pho4', 'pro1', 'art1')
[0.511353711790393, 0.3045582896329165, 0.594956658786446]
619 ('pho3', 'pro1', 'pro3',

913 ('pho1', 'pho2', 'art2', 'art3', 'art4')
[0.5152838427947598, 0.31505250875145857, 0.6382978723404256]
916 ('pho1', 'pho3', 'pho4', 'pro1', 'pro4')
[0.5135371179039301, 0.2837167343256653, 0.49566587864460204]
917 ('pho1', 'pho3', 'pho4', 'pro1', 'art1')
[0.5058951965065502, 0.2876068376068376, 0.5303388494877856]
918 ('pho1', 'pho3', 'pho4', 'pro1', 'art2')
[0.5360262008733624, 0.3228476821192053, 0.6146572104018913]
919 ('pho1', 'pho3', 'pho4', 'pro1', 'art3')
[0.5104803493449782, 0.2898488120950324, 0.52876280535855]
920 ('pho1', 'pho3', 'pho4', 'pro1', 'art4')
[0.5034934497816594, 0.28199566160520606, 0.512214342001576]
922 ('pho1', 'pho3', 'pho4', 'pro2', 'pro4')
[0.5056768558951965, 0.2787905735882614, 0.4940898345153664]
923 ('pho1', 'pho3', 'pho4', 'pro2', 'art1')
[0.503056768558952, 0.29507529507529506, 0.5713159968479118]
924 ('pho1', 'pho3', 'pho4', 'pro2', 'art2')
[0.5102620087336245, 0.31454683929931454, 0.6509062253743105]
927 ('pho1', 'pho3', 'pho4', 'pro3', 'pro4')


1082 ('pho1', 'pro1', 'pro4', 'art2', 'art3')
[0.509825327510917, 0.31216320246343343, 0.6390858944050434]
1083 ('pho1', 'pro1', 'pro4', 'art2', 'art4')
[0.506113537117904, 0.30163803435876946, 0.594956658786446]
1088 ('pho1', 'pro1', 'art2', 'art3', 'art4')
[0.5034934497816594, 0.30363423212192264, 0.6122931442080378]
1113 ('pho1', 'pro3', 'pro4', 'art2', 'art4')
[0.5013100436681223, 0.29932779754843813, 0.5965327029156816]
1120 ('pho1', 'pro4', 'art1', 'art2', 'art4')
[0.508296943231441, 0.31059730250481693, 0.6351457840819543]
1122 ('pho1', 'pro4', 'art2', 'art3', 'art4')
[0.506768558951965, 0.3000807754442649, 0.5855003940110323]
1127 ('pho2', 'pho3', 'pho4', 'pro1', 'art1')
[0.5301310043668123, 0.3123671908202295, 0.5791962174940898]
1128 ('pho2', 'pho3', 'pho4', 'pro1', 'art2')
[0.5021834061135371, 0.31888212110354713, 0.7013396375098503]
1129 ('pho2', 'pho3', 'pho4', 'pro1', 'art3')
[0.505240174672489, 0.306558013193636, 0.6225374310480694]
1138 ('pho2', 'pho3', 'pho4', 'pro3', 

1610 ('pho1', 'pho2', 'pho3', 'pho4', 'art1', 'art4')
[0.5133187772925765, 0.2866666666666667, 0.508274231678487]
1611 ('pho1', 'pho2', 'pho3', 'pho4', 'art2', 'art3')
[0.5323144104803493, 0.3282172373081464, 0.6572104018912529]
1612 ('pho1', 'pho2', 'pho3', 'pho4', 'art2', 'art4')
[0.5314410480349345, 0.31629660661918724, 0.594956658786446]
1613 ('pho1', 'pho2', 'pho3', 'pho4', 'art3', 'art4')
[0.511353711790393, 0.28703296703296705, 0.5145784081954294]
1615 ('pho1', 'pho2', 'pho3', 'pro1', 'pro2', 'pro4')
[0.5050218340611353, 0.28209606986899566, 0.5090622537431048]
1616 ('pho1', 'pho2', 'pho3', 'pro1', 'pro2', 'art1')
[0.5058951965065502, 0.2859603789836348, 0.5232466509062254]
1617 ('pho1', 'pho2', 'pho3', 'pro1', 'pro2', 'art2')
[0.5056768558951965, 0.30902111324376197, 0.6343577620173365]
1621 ('pho1', 'pho2', 'pho3', 'pro1', 'pro3', 'art1')
[0.5163755458515283, 0.293630017452007, 0.5303388494877856]
1622 ('pho1', 'pho2', 'pho3', 'pro1', 'pro3', 'art2')
[0.5006550218340612, 0.310

...

3836 ('pho1', 'pho2', 'pho3', 'pho4', 'pro2', 'pro3', 'pro4', 'art2', 'art4')
[0.5331877729257642, 0.3093462044756472, 0.5555555555555556]
3837 ('pho1', 'pho2', 'pho3', 'pho4', 'pro2', 'pro3', 'pro4', 'art3', 'art4')
[0.5266375545851528, 0.27933235149729996, 0.44838455476753347]
3838 ('pho1', 'pho2', 'pho3', 'pho4', 'pro2', 'pro3', 'art1', 'art2', 'art3')
[0.5305676855895196, 0.32144304823672476, 0.6249014972419228]
3839 ('pho1', 'pho2', 'pho3', 'pho4', 'pro2', 'pro3', 'art1', 'art2', 'art4')
[0.5379912663755458, 0.3178494623655914, 0.582348305752561]
3840 ('pho1', 'pho2', 'pho3', 'pho4', 'pro2', 'pro3', 'art1', 'art3', 'art4')
[0.5338427947598253, 0.2943969610636277, 0.48857368006304175]
3841 ('pho1', 'pho2', 'pho3', 'pho4', 'pro2', 'pro3', 'art2', 'art3', 'art4')
[0.548471615720524, 0.3176631675034231, 0.5484633569739953]
3842 ('pho1', 'pho2', 'pho3', 'pho4', 'pro2', 'pro4', 'art1', 'art2', 'art3')
[0.5373362445414848, 0.32112794612794615, 0.6012608353033885]
3843 ('pho1', 'pho2', 'p

3895 ('pho1', 'pho2', 'pho4', 'pro1', 'pro2', 'art1', 'art2', 'art3', 'art4')
[0.5480349344978166, 0.3204840878529807, 0.5634357762017337]
3896 ('pho1', 'pho2', 'pho4', 'pro1', 'pro3', 'pro4', 'art1', 'art2', 'art3')
[0.5299126637554585, 0.3111111111111111, 0.5736800630417652]
3897 ('pho1', 'pho2', 'pho4', 'pro1', 'pro3', 'pro4', 'art1', 'art2', 'art4')
[0.5366812227074236, 0.311533362792753, 0.5555555555555556]
3898 ('pho1', 'pho2', 'pho4', 'pro1', 'pro3', 'pro4', 'art1', 'art3', 'art4')
[0.5301310043668123, 0.28578360019408056, 0.4641449960598897]
3899 ('pho1', 'pho2', 'pho4', 'pro1', 'pro3', 'pro4', 'art2', 'art3', 'art4')
[0.5414847161572053, 0.30983981693363843, 0.5334909377462569]
3900 ('pho1', 'pho2', 'pho4', 'pro1', 'pro3', 'art1', 'art2', 'art3', 'art4')
[0.5462882096069869, 0.3206208425720621, 0.5697399527186762]
3901 ('pho1', 'pho2', 'pho4', 'pro1', 'pro4', 'art1', 'art2', 'art3', 'art4')
[0.5393013100436681, 0.3118568232662192, 0.5492513790386131]
3902 ('pho1', 'pho2', 'pho

3955 ('pho1', 'pho4', 'pro1', 'pro2', 'pro3', 'pro4', 'art1', 'art3', 'art4')
[0.515938864628821, 0.27808988764044945, 0.46808510638297873]
3956 ('pho1', 'pho4', 'pro1', 'pro2', 'pro3', 'pro4', 'art2', 'art3', 'art4')
[0.5447598253275109, 0.3102325581395349, 0.5256107171000788]
3957 ('pho1', 'pho4', 'pro1', 'pro2', 'pro3', 'art1', 'art2', 'art3', 'art4')
[0.5432314410480349, 0.3168669336893636, 0.5610717100078803]
3958 ('pho1', 'pho4', 'pro1', 'pro2', 'pro4', 'art1', 'art2', 'art3', 'art4')
[0.5384279475982533, 0.3128046078865751, 0.5563435776201734]
3959 ('pho1', 'pho4', 'pro1', 'pro3', 'pro4', 'art1', 'art2', 'art3', 'art4')
[0.5388646288209606, 0.31390728476821195, 0.5602836879432624]
3960 ('pho1', 'pho4', 'pro2', 'pro3', 'pro4', 'art1', 'art2', 'art3', 'art4')
[0.5353711790393013, 0.3095343680709534, 0.5500394011032309]
3961 ('pho1', 'pro1', 'pro2', 'pro3', 'pro4', 'art1', 'art2', 'art3', 'art4')
[0.5133187772925765, 0.29591836734693877, 0.5484633569739953]
3962 ('pho2', 'pho3', 'p

4018 ('pho1', 'pho2', 'pho3', 'pho4', 'pro1', 'pro2', 'pro3', 'pro4', 'art1', 'art3')
[0.5253275109170306, 0.28943694741740345, 0.49014972419227737]
4019 ('pho1', 'pho2', 'pho3', 'pho4', 'pro1', 'pro2', 'pro3', 'pro4', 'art1', 'art4')
[0.5283842794759825, 0.2804337111877772, 0.44838455476753347]
4020 ('pho1', 'pho2', 'pho3', 'pho4', 'pro1', 'pro2', 'pro3', 'pro4', 'art2', 'art3')
[0.5412663755458516, 0.3147818343722173, 0.5571315996847912]
4021 ('pho1', 'pho2', 'pho3', 'pho4', 'pro1', 'pro2', 'pro3', 'pro4', 'art2', 'art4')
[0.5393013100436681, 0.31015801354401806, 0.541371158392435]
4022 ('pho1', 'pho2', 'pho3', 'pho4', 'pro1', 'pro2', 'pro3', 'pro4', 'art3', 'art4')
[0.52882096069869, 0.2824278022515908, 0.45468873128447596]
4023 ('pho1', 'pho2', 'pho3', 'pho4', 'pro1', 'pro2', 'pro3', 'art1', 'art2', 'art3')
[0.5412663755458516, 0.3220701454234388, 0.5933806146572104]
4024 ('pho1', 'pho2', 'pho3', 'pho4', 'pro1', 'pro2', 'pro3', 'art1', 'art2', 'art4')
[0.5486899563318778, 0.3183060

4074 ('pho2', 'pho3', 'pho4', 'pro1', 'pro2', 'pro3', 'pro4', 'art1', 'art3', 'art4')
[0.5203056768558952, 0.2751937984496124, 0.44759653270291566]
4075 ('pho2', 'pho3', 'pho4', 'pro1', 'pro2', 'pro3', 'pro4', 'art2', 'art3', 'art4')
[0.5362445414847161, 0.30344827586206896, 0.5200945626477541]
4076 ('pho2', 'pho3', 'pho4', 'pro1', 'pro2', 'pro3', 'art1', 'art2', 'art3', 'art4')
[0.5364628820960698, 0.3117283950617284, 0.5571315996847912]
4077 ('pho2', 'pho3', 'pho4', 'pro1', 'pro2', 'pro4', 'art1', 'art2', 'art3', 'art4')
[0.5406113537117904, 0.31065759637188206, 0.5397951142631994]
4078 ('pho2', 'pho3', 'pho4', 'pro1', 'pro3', 'pro4', 'art1', 'art2', 'art3', 'art4')
[0.5301310043668123, 0.2992269213278763, 0.5185185185185185]
4079 ('pho2', 'pho3', 'pho4', 'pro2', 'pro3', 'pro4', 'art1', 'art2', 'art3', 'art4')
[0.5351528384279476, 0.30490018148820325, 0.5295508274231678]
4080 ('pho2', 'pho3', 'pro1', 'pro2', 'pro3', 'pro4', 'art1', 'art2', 'art3', 'art4')
[0.5192139737991266, 0.29935

In [90]:
res_df=pd.DataFrame(result,columns=['Feature','Accuracy','Precision','Recall'])

In [91]:
res_df

,Feature,Accuracy,Precision,Recall
0,"('pho2',)",0.633188,0.321460,0.291568
1,"('pro1',)",0.670524,0.144970,0.038613
2,"('pho1', 'pho4')",0.508297,0.293920,0.552403
3,"('pho1', 'art1')",0.506114,0.294580,0.561072
4,"('pho2', 'pho3')",0.657424,0.356870,0.294720
...,...,...,...,...
1970,"('pho1', 'pho2', 'pho3', 'pho4', 'pro2', 'pro3...",0.551528,0.317016,0.535855
1971,"('pho1', 'pho2', 'pho3', 'pro1', 'pro2', 'pro3...",0.541485,0.311394,0.540583
1972,"('pho1', 'pho2', 'pho4', 'pro1', 'pro2', 'pro3...",0.544323,0.308341,0.518519
1973,"('pho1', 'pho3', 'pho4', 'pro1', 'pro2', 'pro3...",0.544760,0.309167,0.520883


In [95]:
res_df.to_csv("/home/vboxuser/VoiceFiles/pca grid feature scores.csv",index=False)

In [86]:
obj1=classification_models()

score=obj1.SVM_model(X,Y,X_test,Y_test)

In [87]:
score

[0.5631004366812227, 0.31952662721893493, 0.5106382978723404]